In [1]:
# SPARK_HOME="/Users/ssv/spark-2.2.0-bin-hadoop2.7"
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
# os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["SPARK_HOME"]='/Users/ssv/spark-2.2.0-bin-hadoop2.7'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

IllegalArgumentException: "Error while instantiating 'org.apache.spark.sql.hive.HiveSessionStateBuilder':"

# MLLib algorithms

** A Few Useful Things to Know about Machine Learning, Pedro Domingos  **

https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf

LEARNING = REPRESENTATION + EVALUATION + OPTIMIZATION

** Representation. ** A classifier must be represented in some
formal language that the computer can handle. Conversely,
choosing a representation for a learner is tantamount
to choosing the set of classifiers that it can
possibly learn. This set is called the hypothesis space
of the learner. If a classifier is not in the hypothesis
space, it cannot be learned. A related question, which
we will address in a later section, is how to represent
the input, i.e., what features to use.

** Evaluation. ** An evaluation function (also called objective
function or scoring function) is needed to distinguish
good classifiers from bad ones. The evaluation function
used internally by the algorithm may differ from the
external one that we want the classifier to optimize, for
ease of optimization (see below) and due to the issues
discussed in the next section

** Optimization. ** Finally, we need a method to search among
the classifiers in the language for the highest-scoring
one. The choice of optimization technique is key to the
efficiency of the learner, and also helps determine the
classifier produced if the evaluation function has more
than one optimum. It is common for new learners to
start out using off-the-shelf optimizers, which are later
replaced by custom-designed ones.

## Regression

http://spark.apache.org/docs/latest/mllib-classification-regression.html

linear least squares, Lasso, ridge regression, decision trees, random forests, gradient-boosted trees, isotonic regression`

Mathematical formulation

http://spark.apache.org/docs/latest/mllib-linear-methods.html#mathematical-formulation

Optimization

http://spark.apache.org/docs/latest/mllib-optimization.html 

In [2]:
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD
from numpy import array
import random

### Simple example ###

In [3]:
data = [
     LabeledPoint(4.0, [0.0]),
     LabeledPoint(5.0, [1.0]),
     LabeledPoint(6.0, [2.0]),
     LabeledPoint(7.0, [3.0])
]

In [4]:
lrm = LinearRegressionWithSGD.train(sc.parallelize(data))

/usr/hdp/current/spark2-client/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [5]:
lrm

(weights=[2.71442584085], intercept=0.0)

In [6]:
lrm.predict([0.0])

0.0

In [7]:
lrm.predict([3.0])

8.1432775225638405

In [8]:
lrm.predict([1.0])

2.7144258408546134

In [9]:
lrm = LinearRegressionWithSGD.train(sc.parallelize(data), \
                                    initialWeights=array([1.0]), intercept=True)

/usr/hdp/current/spark2-client/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [10]:
lrm

(weights=[1.03422300001], intercept=3.9269324708655176)

In [11]:
lrm.predict([0.0])

3.9269324708655176

In [12]:
lrm.predict([20.0])

24.611392471097115

### more complex example ###

In [13]:
points = sc.parallelize(range(100000)).map(lambda x: \
                LabeledPoint(x*25+17+random.uniform(-10,10), [float(x)]))

In [14]:
points.takeSample(False, 10)

[LabeledPoint(619950.3479960456, [24797.0]),
 LabeledPoint(2086926.5536741477, [83476.0]),
 LabeledPoint(299975.6761833584, [11998.0]),
 LabeledPoint(1740589.654409388, [69623.0]),
 LabeledPoint(1510133.0971272558, [60405.0]),
 LabeledPoint(138538.96102203603, [5541.0]),
 LabeledPoint(2356742.4493169007, [94269.0]),
 LabeledPoint(803443.8290466962, [32137.0]),
 LabeledPoint(504000.9787051513, [20159.0]),
 LabeledPoint(5670.861633887015, [226.0])]

In [15]:
model = LinearRegressionWithSGD.train(points, intercept=True)

/usr/hdp/current/spark2-client/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [16]:
model

(weights=[nan], intercept=nan)

how to determine what happened?

...

...

In [17]:
model = LinearRegressionWithSGD.train(points, iterations=10, step=1e-9, intercept=True)

/usr/hdp/current/spark2-client/python/pyspark/mllib/regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [18]:
model

(weights=[24.9978325067], intercept=1.0003749894161624)

https://issues.apache.org/jira/browse/SPARK-5273

https://issues.apache.org/jira/browse/SPARK-1859